### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 7.7.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import norm
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run single_alpha_decay_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    alpha = params[0]
    beta = params[1]
    decay = params[2]

    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], alpha)
        # then update value
        EVs[t] = update_value(Probs[t]) 
        
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(t)
        for x in options:
            Probs[x] = Probs[x] + decay * (0.5 - Probs[x])
            EVs[x] =  update_value(Probs[x]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
# check simulation 
#sim_data(15,[.2, 2, .2])

### Create set of parameters
- distribution of alpha and beta parameters modeled following Cutler et al., 2021
- distribution of decay parameters based on mean/SD reported for RL model in Collins & Frank, 2014

In [20]:
alphas = beta.rvs(a = 1.1, b = 1.1, size = 150)
betas = gamma.rvs(a = 1.2, scale = 5, size = 150)
decays = norm.rvs(loc = 0.096, scale = 0.21, size = 150)
decays[decays<0] = 0
decays[decays>1] = 1
#print(decays)

### Simulate data

In [24]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha', 'Beta', 'Decay'])
for p in range(len(alphas)):
    dt = sim_data(15, [alphas[p], betas[p], decays[p]])
    dt['Subject'] = [p + 1] * 45
    dt['Alpha'] = [alphas[p]] * 45
    dt['Beta'] = [betas[p]] * 45
    dt['Decay'] = [decays[p]] * 45
    dt = pd.DataFrame(dt)
    data = pd.concat([data, dt])

data.to_csv(path_or_buf = '../../output/simulation/sim_1alpha_decay_model_data.csv', index = False)

In [25]:
# a = pd.unique(data['Subject'])
# print(a)

[1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150]


### New function to fit model to multiple subjects

In [26]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'alpha', 'beta', 'decay'])
    a = pd.unique(data['Subject'])
    print('Number of Subs: ', len(a))
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==sub]
        dt = df.to_dict()
        #print('Data: ', dt)
        params = model_fit(dt)
        #print(params)
        line = {'Subject': sub, 'alpha': params.x[0], 'beta': params.x[1], 'decay': params.x[2]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [27]:
precover = model_fit_mult_subject(data)

Number of Subs:  150
Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9
Subject:  10
Subject:  11
Subject:  12
Subject:  13
Subject:  14
Subject:  15
Subject:  16
Subject:  17
Subject:  18
Subject:  19
Subject:  20
Subject:  21
Subject:  22
Subject:  23
Subject:  24
Subject:  25
Subject:  26
Subject:  27
Subject:  28
Subject:  29
Subject:  30
Subject:  31
Subject:  32
Subject:  33
Subject:  34
Subject:  35
Subject:  36
Subject:  37
Subject:  38
Subject:  39
Subject:  40
Subject:  41
Subject:  42
Subject:  43
Subject:  44
Subject:  45
Subject:  46
Subject:  47
Subject:  48
Subject:  49
Subject:  50
Subject:  51
Subject:  52
Subject:  53
Subject:  54
Subject:  55
Subject:  56
Subject:  57
Subject:  58
Subject:  59
Subject:  60
Subject:  61
Subject:  62
Subject:  63
Subject:  64
Subject:  65
Subject:  66
Subject:  67
Subject:  68
Subject:  69
Subject:  70
Subject:  71
Subject:  72
Subject:  73
Subject:  74
Subject:  75
Subject:  76


In [28]:
print(precover)

     Subject     alpha       beta     decay
0        1.0  1.000000   9.686765  0.000000
1        2.0  1.000000  12.988219  0.000000
2        3.0  0.516835   0.652691  1.000000
3        4.0  1.000000  17.386313  0.000000
4        5.0  0.017526  20.000000  0.326259
5        6.0  1.000000  15.252588  0.000000
6        7.0  1.000000  11.522223  0.000000
7        8.0  1.000000  13.350251  0.000000
8        9.0  1.000000  12.566541  0.000000
9       10.0  1.000000   9.000535  0.000000
10      11.0  1.000000   0.273618  0.236678
11      12.0  1.000000  10.875076  0.000000
12      13.0  1.000000  11.925131  0.000000
13      14.0  1.000000  20.000000  0.702221
14      15.0  1.000000  14.853214  0.000000
15      16.0  1.000000  12.768675  0.000000
16      17.0  1.000000   9.824844  0.000000
17      18.0  0.229564  20.000000  0.400431
18      19.0  1.000000   4.664944  0.000000
19      20.0  1.000000  17.551527  0.000000
20      21.0  0.088699  20.000000  0.000000
21      22.0  1.000000  11.38896

In [29]:
precover.to_csv(path_or_buf = '../../output/simulation/sim_1alpha_decay_model_fits.csv', index = False)